In [ ]:
# default_exp text.text_preprocessing

# Text Preprocessing

> Module for preprocessing text. 

In [ ]:
# export
from AugmenTF.core import *
from AugmenTF.basic_data import *
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow import strings, py_function
import tensorflow as tf
import re
import html
from string import punctuation
import spacy
from spacy.symbols import ORTH
import sentencepiece as sps

In [ ]:
# export

_all_ = [
    "BOS",
    "EOS",
    "FLD",
    "UNK",
    "PAD",
    "TK_MAJ",
    "TK_UP",
    "TK_REP",
    "TK_WREP",
    "STEPS",
]

BOS = 'xxbos'
EOS = 'xxeos'
FLD = 'xxfld'
UNK = 'xxunk'
PAD = 'xxpad'
TK_MAJ = 'xxmaj'
TK_UP = 'xxup'
TK_REP = 'xxrep'
TK_WREP = 'xxwrep'

def spec_add_spaces(t:str) -> str:
    "Add spaces around / and # in `t`. \n"
    return re.sub(r'([/#\n])', r' \1 ', t)

def separate_punctuation(string):
    '''
    Adds spaces around all punctuation.
    '''
    return re.sub(f'([{punctuation}\n])', r' \1 ',
                  string).strip()

def rm_useless_spaces(t:str) -> str:
    "Remove multiple spaces in `t`."
    return re.sub(' {2,}', ' ', t)

def replace_rep(string):
    "Replace repetitions at the character level in `string`."
    def _replace_rep(m):
        c,cc = m.groups()
        return f' {TK_REP} {len(cc)+1} {c} '
    re_rep = re.compile(r'(\S)(\1{3,})')
    return re_rep.sub(_replace_rep, string)

def replace_wrep(t):
    "Replace word repetitions in `t`."
    def _replace_wrep(m):
        c,cc = m.groups()
        return f' {TK_WREP} {len(cc.split())+1} {c} '
    re_wrep = re.compile(r'(\b\w+\W+)(\1{3,})')
    return re_wrep.sub(_replace_wrep, t)

def fix_html(x):
    "List of replacements from html strings in `x`."
    re1 = re.compile(r'  +')
    
    x = x.replace('#39;', "'").replace('amp;', '&').replace('#146;', "'").replace(
        'nbsp;', ' ').replace('#36;', '$').replace('\\n', "\n").replace('quot;', "'").replace(
        '<br />', "\n").replace('\\"', '"').replace('<unk>',UNK).replace(' @.@ ','.').replace(
        ' @-@ ','-').replace(' @,@ ',',').replace('\\', ' \\ ')
    return re1.sub(' ', html.unescape(x))

def replace_all_caps(x):
    "Replace tokens in ALL CAPS in `x` by their lower version and add `TK_UP` before."
    res = []
    for t in x:
        if t.isupper() and len(t) > 1: res.append(TK_UP); res.append(t.lower())
        else: res.append(t)
    return ''.join(res)

def deal_caps(x):
    "Replace all Capitalized tokens in `x` by their lower version and add `TK_MAJ` before."
    res = []
    for t in x:
        if t == '': continue
        if t[0].isupper() and len(t) > 1 and t[1:].islower(): res.append(TK_MAJ)
        res.append(t.lower())
    return ''.join(res)

def fix_ascii(string):
    '''
    Fixes any ascii characters that may be in the text.
    '''
    return string.encode('ascii', errors='ignore').decode('utf-8')

STEPS = [fix_html, replace_rep, replace_wrep, separate_punctuation, rm_useless_spaces, replace_all_caps, deal_caps]

def preprocess_text(string, funcs=STEPS):
    '''
    General function for cleaning text.
    Applites functions in order to a string.
    
    Options:
    
    string: string you want to process for tokenization.
    funcs: list, series of ordered functions you'd like to apply to string.
    
    Returns:
    
    string, text that's been processed by funcs.
    '''
    return apply_chained_funcs(string, funcs).strip()

These functions help clean text. They are largely taken from fastai.text.transform. They perform some basic text cleaning functionality.

In [ ]:
preprocess_text('''
Hello!\nThis is an example of how to #clean text up a bit...
aaaaa AAA
''')

'hello ! \n this is an example of how to # clean text up a bit . . . \n xxrep 5 a aaa'

In [ ]:
# Export
class BaseTokenizer():
    "Basic class for a tokenizer function."
    def __init__(self, lang):
        self.lang = lang
    def tokenizer(self, t):
        return t.split(' ')
    def add_special_cases(self, toks):
        pass

class SpacyTokenizer(BaseTokenizer):
    "Wrapper around a spacy tokenizer to make it a `BaseTokenizer`."
    def __init__(self, lang):
        self.tok = spacy.blank(lang, disable=["parser","tagger","ner"])

    def tokenizer(self, t):
        return [t.text for t in self.tok.tokenizer(t)]

    def add_special_cases(self, tokens):
        for w in tokens:
            self.tok.tokenizer.add_special_case(w, [{ORTH: w}])
            
class SentencePieceTokenizer():
    pass

In [ ]:
trainer = sps.SentencePieceTrainer.Train()

TypeError: SentencePieceTrainer_Train() takes exactly 1 argument (0 given)

Blockers:
* regex does not support backreferences. This prevents implementation of some core functionality as a tf.function.